# Backtesting Analysis
This project is to demonstrate how to utilize backtesting to assess stock performance, further increasing the likelihood and filter down to the best stock candidates. 

## What is Backtesting?
Backtesting with Simple Moving Averages (SMAs) in Python is a good step in evaluating the effectiveness of an investment strategy before risking real money in the financial markets. This approach use simulating trades using historical price data to assess how a strategy would have performed in the past. 

By applying these rules to historical price data, key performance metrics can be measured, such as returns, drawdowns, and risk-adjusted ratios. These results provide valuable insights into the strategy's historical profitability and risk profile, helping traders make informed decisions. To summarize, backtesting gives a sort of conclusion to investors where `"if a stock is performing well in the past, it is more likely to perform well in the future.`

This Backtesting Analysis can be done with the use of a library called `Backtesting.py` which simplifies the process by passing the stock listing of choice and its history, the number of cash to invest, and the method of strategy to be used as an argument.

In [12]:
# import libraries
from datetime import datetime
from pandas_datareader import data as pdr
import yfinance as yf
import pandas as pd
import time
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import random
import warnings
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [13]:
df = pd.read_csv('exports/filtered_class_a_b.csv')

## Part I: Singular Ticker Testing
Retrieving the stock symbol candidate historical prices (yfinance)
The Backtesting.py library requires a "bring your own data" approach which can be easily retrieved through Yahoo Finance with yfinance API. The historical prices are retrieved which will be used to be analyzed. 

To demonstrate, one of the potential candidates that successfully went through the filtration by its dividend and foundational health (class A or B) will be used for this backtesting. A 1 year period is simulated for this demonstration.

In [14]:
# Example OHLC daily data for Google Inc.
import backtesting
from backtesting.test import GOOG
backtesting.set_bokeh_output(notebook=False)

ticker = yf.Ticker("5248.KL")

In [15]:
# Get historical data for the ticker
hist = ticker.history(period="1y")
print(hist)

                               Open      High       Low     Close    Volume  \
Date                                                                          
2022-11-08 00:00:00+08:00  1.799460  1.799460  1.762923  1.762923   1266600   
2022-11-09 00:00:00+08:00  1.753789  1.808595  1.753789  1.781192   2903000   
2022-11-10 00:00:00+08:00  1.781192  1.808595  1.762923  1.781192   1767200   
2022-11-11 00:00:00+08:00  1.790326  1.790326  1.753789  1.762923   1036800   
2022-11-14 00:00:00+08:00  1.762923  1.772058  1.753789  1.772058   1122700   
2022-11-15 00:00:00+08:00  1.772057  1.799460  1.753789  1.762923   1998400   
2022-11-16 00:00:00+08:00  1.762923  1.762923  1.735520  1.735520   1942700   
2022-11-17 00:00:00+08:00  1.735520  1.772058  1.735520  1.772058    799300   
2022-11-21 00:00:00+08:00  1.753789  1.753789  1.726386  1.726386   2805100   
2022-11-22 00:00:00+08:00  1.744654  1.753789  1.735520  1.744654   1293200   
2022-11-23 00:00:00+08:00  1.735520  1.762923  1.735

## Defining Backtesting Strategy with Simple Moving Average

This Python backtesting code implements a simple moving average (SMA) crossover trading strategy using the Backtesting library. In this strategy, two SMAs are defined: a shorter-term SMA (n1) and a longer-term SMA (n2). The key idea is to use these SMAs to identify potential buy and sell signals based on their crossovers.


- When the shorter-term SMA (sma1) crosses above the longer-term SMA (sma2), it triggers a buy signal. Any existing positions (long or short) are closed, and a long position (buy) is initiated.
- Conversely, when sma1 crosses below sma2, it triggers a sell signal. Again, existing positions are closed, and a short position (sell) is initiated.

This strategy aims to capture potential trends in the asset's price movements by leveraging SMA crossovers. Backtesting is then used to evaluate the strategy's historical performance based on these rules, providing insights into its profitability and effectiveness.

In [16]:
import pandas as pd


def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()

In [31]:
from backtesting import Strategy
from backtesting.lib import crossover


class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 0
    n2 = 0
    
    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    
    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            #self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            #self.sell()

## Running the Backtest

In [32]:
from backtesting import Backtest

bt = Backtest(hist, SmaCross, cash=10000, commission=.002)
stats = bt.run()
stats

Start                     2022-11-08 00:00...
End                       2023-11-08 00:00...
Duration                    365 days 00:00:00
Exposure Time [%]                         0.0
Equity Final [$]                      10000.0
Equity Peak [$]                       10000.0
Return [%]                                0.0
Buy & Hold Return [%]               41.809937
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     0.0
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                    

## Analyzing the Results

- Return [%]: This is the total return on your investment as a percentage. It shows how much your initial capital has grown or shrunk.


- Buy & Hold Return [%]: This is the return you would have achieved if you simply bought and held the asset without any trading. It provides a benchmark for comparison.


- Volatility (Ann.) [%]: This represents the annualized volatility of your strategy. It measures the variation in returns over time. Lower volatility is generally preferred.


- Sharpe Ratio: The Sharpe Ratio measures the risk-adjusted return of your strategy. A higher Sharpe Ratio indicates better risk-adjusted performance. It's a common metric for assessing investment strategies.


- Sortino Ratio: Similar to the Sharpe Ratio, the Sortino Ratio measures risk-adjusted return but focuses on downside risk (volatility of negative returns). A higher Sortino Ratio indicates better risk-adjusted performance, especially in strategies that aim to minimize downside risk.


- Calmar Ratio: The Calmar Ratio measures the risk-adjusted return relative to the maximum drawdown. A higher Calmar Ratio is generally better, as it indicates better returns relative to the risk of significant losses.


- Win Rate [%]: This is the percentage of profitable trades out of the total number of trades. A higher win rate is generally preferred.


- Avg. Trade [%]: This is the average percentage return of all trades in your strategy. It provides a measure of the typical trade performance.


- Avg. Trade Duration: This is the average duration (in days) of all trades.


- Profit Factor: The Profit Factor measures the ratio of gross profit to gross loss. A higher profit factor is generally desirable.


- Expectancy [%]: Expectancy represents the average amount you can expect to win (or lose) per trade.

# Interpreting the Results:

Ideal ranges for backtesting metrics can vary depending on your specific trading strategy, risk tolerance, and investment goals. However, I can provide some general guidelines for what traders often consider ideal or acceptable ranges for key backtesting metrics in a 12-month backtest. Keep in mind that these ranges are not set in stone, and what is considered ideal can differ from one trader to another.

1. **Return [%]:**
   - Ideal Range: Positive returns are typically desirable. Aiming for a return that outperforms a benchmark (e.g., Buy & Hold) is a common goal.

2. **Sharpe Ratio:**
   - Ideal Range: A Sharpe Ratio greater than 1 is often considered good, indicating a positive risk-adjusted return. Higher values are better.

3. **Sortino Ratio:**
   - Ideal Range: A Sortino Ratio greater than 1 is generally seen as positive. It focuses on minimizing downside risk, so a higher Sortino Ratio is preferred.

4. **Calmar Ratio:**
   - Ideal Range: A Calmar Ratio greater than 1 is typically desired. It assesses risk-adjusted return relative to maximum drawdown. Higher values are better.

5. **Max. Drawdown [%]:**
   - Ideal Range: A lower maximum drawdown is better. Ideally, it should be less than 20% for most traders, but this can vary based on risk tolerance.

6. **Volatility (Ann.) [%]:**
   - Ideal Range: Lower volatility is generally preferred. Volatility should be consistent with your risk tolerance and investment goals.

7. **Win Rate [%]:**
   - Ideal Range: A win rate above 50% is typically desirable. A higher win rate indicates more winning trades.

8. **Profit Factor:**
   - Ideal Range: A profit factor greater than 1 is usually considered good. It indicates that the strategy's gross profit outweighs gross losses.

10. **Expectancy [%]:**
    - Ideal Range: A positive expectancy indicates that, on average, each trade contributes positively to the strategy's performance.

Note that these ranges are general guidelines and may vary based on the specific trading strategy and risk tolerance. Some traders may be willing to accept higher drawdowns in exchange for potentially higher returns, while others prioritize lower risk and stability.

Additionally, backtesting is just one part of the evaluation process. Real-world trading involves factors like slippage, transaction costs, and market conditions that can impact performance. It's essential to consider these factors and conduct thorough analysis before implementing a trading strategy in live markets.

In [33]:
from bokeh.plotting import show

# Assuming you have created the 'bt' plot object
bt.plot()



Row(id='2369', ...)

# Part II: Bulk Ticker Testing & Export to CSV

In [2]:
df = pd.read_csv('exports/filtered_class_a_b.csv')

In [10]:

results = []

# Loop through each ticker symbol in the CSV file
for ticker_symbol in df['Ticker']:
    # Construct the ticker object
    ticker = yf.Ticker(ticker_symbol)

    # Get historical data for the ticker
    hist = ticker.history(period="1y")

    # Perform backtesting for the ticker
    bt = Backtest(hist, SmaCross, cash=10000, commission=.002)
    stats = bt.run()

    # Extract the selected columns from the stats DataFrame
    selected_stats = stats[['Return [%]', 'Buy & Hold Return [%]', 'Return (Ann.) [%]', 
                            'Volatility (Ann.) [%]', 'Avg. Drawdown [%]', 'Win Rate [%]', 
                            'Avg. Trade [%]', 'Profit Factor', 'Expectancy [%]']]

    # Convert selected_stats to a DataFrame with a single row
    selected_stats_df = selected_stats.to_frame().T

    # Reset the index to make it a row
    selected_stats_df = selected_stats_df.reset_index(drop=True)

    # Rename the columns
    selected_stats_df.columns = ['Return [%]', 'Buy & Hold Return [%]', 'Return (Ann.) [%]',
                                'Volatility (Ann.) [%]', 'Avg. Drawdown [%]', 'Win Rate [%]',
                                'Avg. Trade [%]', 'Profit Factor', 'Expectancy [%]']

    # Append the results to the list along with the Ticker symbol
    results.append({
        'Ticker': ticker_symbol,
        **selected_stats_df.iloc[0].to_dict(),  # Use ** to merge dictionaries
    })

# Convert the results list to a DataFrame
results_df = pd.DataFrame(results)

# Now, you have the backtesting results for each ticker in the CSV file


In [11]:
results_df

,Ticker,Return [%],Buy & Hold Return [%],Return (Ann.) [%],Volatility (Ann.) [%],Avg. Drawdown [%],Win Rate [%],Avg. Trade [%],Profit Factor,Expectancy [%]
0,7090.KL,6.416438,12.325211,6.633644,21.922503,-3.843551,54.545455,0.567139,1.533441,0.658037
1,5248.KL,-13.155241,41.809927,-13.607735,19.111162,-14.233281,15.384615,-1.063990,0.698382,-0.842703
2,2062.KL,-25.095244,9.485007,-25.892597,19.281929,-8.672751,35.714286,-2.042905,0.226853,-1.939413
3,6139.KL,10.929217,12.539249,11.356178,16.295912,-2.677627,75.000000,1.305223,9.370113,1.317938
